<a href="https://colab.research.google.com/github/parth-mango/EVA5-Assignments/blob/main/EVA_A5F3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target
  * Reduce the no of parameters below 10k
  * Add dropout to reduce over fitting
  * Adding 1x1 kernels for reducing the no of channles

#Result
  * Parameters - 7,216
  * Performance - 99.32%

#Analysis
  * We have fixed overfitting using the dropout
  * The performance is still below the target 99.4%
  * Upon observing the data we believe introducing rotation transform can improve the performance
  * We have reduced the number of parameters below 10k


In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
def calc_rf(jin, k, rin):
    rout = rin + ((k-1) * jin)
    return rout

calc_rf(4,1,40)

40

In [ ]:
dropout_value= 0.06

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),  #28x28x1 > 28x28x8 : RF: 3x3
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value),
            nn.Conv2d(8, 12, 3, padding=1), # 28x28x8 > 28x28x12 : RF: 5x5
            nn.ReLU(),
            nn.BatchNorm2d(12), 
            nn.Dropout(dropout_value),
            #transition block
            nn.Conv2d(12, 6, 1),            # 28x28x12 > 28x28x6  : RF: 5x5
            nn.MaxPool2d(2,2),              # 28x28x6 > 14x14x6   : RF: 6x6
            nn.Conv2d(6, 12, 3),            # 14x14x6  > 12x12x12 : RF: 10x10
            nn.ReLU(),
            nn.BatchNorm2d(12), 
            nn.Dropout(dropout_value),
            nn.Conv2d(12, 12, 3),           # 12x12x12 > 10x10x12 : RF: 14x14
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value),
            nn.Conv2d(12, 12, 3, padding= 1),# 10x10x12  > 10x10x12 : RF: 18x18
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value),
            nn.MaxPool2d(2,2),               # 10x10x12 > 5x5x12    : RF: 20x20
            nn.Conv2d(12, 12, 3),            # 5x5x12 > 3x3x12      : RF: 28x28
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value),
            nn.Conv2d(12, 12, 3, padding= 1),# 3x3x12 > 3x3x12      : RF: 36x36
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(dropout_value),
            nn.AvgPool2d(3, 2),              # 3x3x12 > 1x1x12 : RF: 40x40
            nn.Conv2d(12, 10 , 1)            # 1x1x12 > 1x1x10 : RF: 40x40
        )
 
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]             876
              ReLU-6           [-1, 12, 28, 28]               0
       BatchNorm2d-7           [-1, 12, 28, 28]              24
           Dropout-8           [-1, 12, 28, 28]               0
            Conv2d-9            [-1, 6, 28, 28]              78
        MaxPool2d-10            [-1, 6, 14, 14]               0
           Conv2d-11           [-1, 12, 12, 12]             660
             ReLU-12           [-1, 12, 12, 12]               0
      BatchNorm2d-13           [-1, 12, 12, 12]              24
          Dropout-14           [-1, 12,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    y_pred = model(data)
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)
    loss.backward()
    optimizer.step()
    pred = y_pred.argmax(dim=1, keepdim=True)  
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))
    return test_loss

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)
    print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Loss=0.12927310168743134 Batch_id=468 Accuracy=90.47: 100%|██████████| 469/469 [00:17<00:00, 27.50it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0741, Accuracy: 9774/10000 (97.74%)

Epoch-1 lr: 0.01


Loss=0.13241180777549744 Batch_id=468 Accuracy=97.52: 100%|██████████| 469/469 [00:16<00:00, 27.91it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0798, Accuracy: 9741/10000 (97.41%)

Epoch-2 lr: 0.01


Loss=0.06514988839626312 Batch_id=468 Accuracy=98.06: 100%|██████████| 469/469 [00:16<00:00, 27.66it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0229, Accuracy: 9926/10000 (99.26%)

Epoch-13 lr: 0.01


Loss=0.06753173470497131 Batch_id=468 Accuracy=99.22: 100%|██████████| 469/469 [00:15<00:00, 30.14it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0210, Accuracy: 9934/10000 (99.34%)

Epoch-14 lr: 0.01


Loss=0.013729152269661427 Batch_id=468 Accuracy=99.11: 100%|██████████| 469/469 [00:15<00:00, 30.37it/s]



Test set: Average loss: 0.0222, Accuracy: 9932/10000 (99.32%)

Epoch-15 lr: 0.01
